<a href="https://colab.research.google.com/github/leolani/cltl-face-all/blob/master/examples/colab/1.visual-features-extraction-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
!ls drive/MyDrive/MELD/MELD.Raw/train/train_splits/ | wc -l
!ls drive/MyDrive/MELD/MELD.Raw/dev/dev_splits_complete/ | wc -l
!ls drive/MyDrive/MELD/MELD.Raw/test/output_repeated_splits_test/ | wc -l

9989
1112
2747


# Clone the repo, install the package, download data, and extract them.

Don't run `pip install -r requirements.txt` from the original installation guide. They are made for python3.7, but colab still uses python3.6. We'll install the necessary packages here on the fly.

In [ ]:
%cd /content
!git clone https://github.com/leolani/cltl-face-all
!cd cltl-face-all/ && pip install .
!pip install omegaconf==2.0.5
!pip install tqdm av

# Download the annotations and stuff
!wget https://raw.githubusercontent.com/leolani/cltl-face-all/master/examples/smaller-datasets-jsons/datasets.json

!gdown --id 16ck7plW9v9eSHGCs5wuB2AhhufPRt3Wi
!unzip smaller-dataset.zip
!rm smaller-dataset.zip
!ls smaller-dataset/  |  wc -l

# Assess the data

The number of videos should be the same

In [2]:
import json

with open('datasets.json', 'r') as stream:
    datasets = json.load(stream)

datasets = datasets['large']

# Let's check out a couple of videos to see if our labeling is correct.

In [4]:
from glob import glob
import random
import os
import json
import numpy as np
import av
import matplotlib.pyplot as plt
import cv2
import random
from IPython.display import HTML
from base64 import b64encode

VIDEOS_PATH = "smaller-dataset/"

diautt = random.choice([foo for foo, bar in datasets['train'].items()])
annot = datasets['train'][diautt]
print(annot)
vid_path = os.path.join(VIDEOS_PATH, diautt) +  '.mp4'

mp4 = open(vid_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width=1280 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

{'SrNo': '4828', 'Utterance': "Are you're hands still wet?", 'Speaker': 'Ross', 'Emotion': 'neutral', 'Sentiment': 'neutral', 'Dialogue_ID': '492', 'Utterance_ID': '1', 'Season': '1', 'Episode': '22', 'StartTime': '0:19:29,042', 'EndTime': '0:19:30,210'}


# Instantiate the three classes we need.

In [5]:
from cltl_face_all.face_alignment import FaceDetection
from cltl_face_all.arcface import ArcFace
from cltl_face_all.agegender import AgeGender

fd = FaceDetection(device='cuda', face_detector='sfd')
ag = AgeGender(device='cpu')
af = ArcFace(device='cpu')

[*] load ckpt from /usr/local/lib/python3.6/dist-packages/cltl_face_all/arcface/./pretrained_models/arc_res50/e_8_b_40000.ckpt


# Run over the images of smaller dataset

In [ ]:
from tqdm.notebook import tqdm
import av
import numpy as np
import os
import shutil

# shutil.rmtree('visual-features', ignore_errors=True)
os.makedirs('drive/MyDrive/MELD/visual-features/smaller-dataset', exist_ok=True)
VIDEOS_PATH = "smaller-dataset/"

for vidpath in tqdm(glob(os.path.join(VIDEOS_PATH, '*.mp4'))):
    # print(vidpath)

    diautt = os.path.basename(vidpath)

    save_full_path  = os.path.join('drive/MyDrive/MELD/visual-features/smaller-dataset', f"{diautt.replace('.mp4', '.npy')}")
    if os.path.isfile(save_full_path) and os.path.getsize(save_full_path) > 512:
        continue

    to_dump = {}
    container = av.open(vidpath)


    for idx, frame in enumerate(container.decode(video=0)):
        to_dump[idx] = []
        numpy_RGB = np.array(frame.to_image())
        batch = numpy_RGB[np.newaxis, ...]
        bboxes = fd.detect_faces(batch)
        # print(f"number of faces detected in the frame {idx} is {len(bboxes[0])}")

        if len(bboxes[0]) == 0:
            continue

        landmarks = fd.detect_landmarks(batch, bboxes)

        faces = fd.crop_and_align(batch, bboxes, landmarks)
        faces = np.concatenate(faces, axis=0)
        embeddings = af.predict(faces)
        ages, genders = ag.predict(faces)

        # print(len(bboxes[0]), len(landmarks[0]), len(ages), len(genders), len(embeddings), len(faces))
        for bb, lm, a, g, emb in zip(bboxes[0], landmarks[0], ages, genders, embeddings):
            x1, y1, x2, y2, prob = bb

            to_append = {'bbox': bb, 
                        'landmark': lm, 
                        'age': a,
                        'gender': g,
                        'embedding': emb}

            to_dump[idx].append(to_append)
        # print(len(to_dump), idx+1)

    np.save(save_full_path, to_dump)

/usr/local/lib/python3.6/dist-packages/cltl_face_all/face_alignment/matlab_cp2tform.py:84: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  r, _, _, _ = lstsq(X, U)


# Run over the images of the entire datasets

In [2]:
import os
from glob import glob
import csv
import json
from tqdm.notebook import tqdm

SAVE_AT = "drive/MyDrive/MELD/visual-features/entire-dataset"

!wget https://raw.githubusercontent.com/declare-lab/MELD/master/data/MELD/train_sent_emo.csv
!wget https://raw.githubusercontent.com/declare-lab/MELD/master/data/MELD/dev_sent_emo.csv
!wget https://raw.githubusercontent.com/declare-lab/MELD/master/data/MELD/test_sent_emo.csv

annotation_path = {}
annotation_path['train'] = "train_sent_emo.csv" 
annotation_path['dev'] = "dev_sent_emo.csv" 
annotation_path['test'] = "test_sent_emo.csv" 

        
VIDS_DIR = {}
VIDS_DIR['train'] = "drive/MyDrive/MELD/MELD.Raw/train/train_splits/" 
VIDS_DIR['dev'] = "drive/MyDrive/MELD/MELD.Raw/dev/dev_splits_complete/" 
VIDS_DIR['test'] = "drive/MyDrive/MELD/MELD.Raw/test/output_repeated_splits_test/"


if not os.path.isfile(os.path.join(SAVE_AT, 'datasets.json')):
    annotations = {}
    for DATATYPE in ['train', 'dev', 'test']:
        with open(annotation_path[DATATYPE]) as f:
            reader = csv.reader(f)
            annotations[DATATYPE] = list(reader)

    datasets = {}
    weird = 0
    for DATATYPE in tqdm(['train', 'dev', 'test']):
        datasets[DATATYPE] = {}
        for row in tqdm(annotations[DATATYPE][1:]):
            SrNo, Utterance, Speaker, Emotion, Sentiment, Dialogue_ID,\
                Utterance_ID, Season, Episode, StartTime, EndTime = row

            if f"dia{Dialogue_ID}_utt{Utterance_ID}.mp4" not in str(os.listdir(VIDS_DIR[DATATYPE])):
                weird+=1
                continue
            datasets[DATATYPE][f"dia{Dialogue_ID}_utt{Utterance_ID}"] = \
                {'SrNo': SrNo, 
                'Utterance': Utterance, 
                'Speaker': Speaker, 
                'Emotion': Emotion, 
                'Sentiment': Sentiment, 
                'Dialogue_ID': Dialogue_ID,
                'Utterance_ID': Utterance_ID, 
                'Season': Season, 
                'Episode': Episode, 
                'StartTime': StartTime, 
                'EndTime':EndTime}
                    
    print(f"the number of video that didn't match is : {weird}")


    with open(os.path.join(SAVE_AT, 'datasets.json'), 'w', encoding='utf8') as stream:
        json.dump(datasets, stream, ensure_ascii=False)

else:
    with open(os.path.join(SAVE_AT, 'datasets.json'), 'r', encoding='utf8') as stream:
        datasets = json.load(stream)

--2020-12-06 11:49:04--  https://raw.githubusercontent.com/declare-lab/MELD/master/data/MELD/train_sent_emo.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1119290 (1.1M) [text/plain]
Saving to: ‘train_sent_emo.csv.2’

train_sent_emo.csv. 100%[===================>]   1.07M  --.-KB/s    in 0.02s   

2020-12-06 11:49:04 (43.7 MB/s) - ‘train_sent_emo.csv.2’ saved [1119290/1119290]

--2020-12-06 11:49:04--  https://raw.githubusercontent.com/declare-lab/MELD/master/data/MELD/dev_sent_emo.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12

In [3]:
from glob import glob
import random
import os
import json
import numpy as np
import av
import matplotlib.pyplot as plt
import cv2
import random
from IPython.display import HTML
from base64 import b64encode

VIDEOS_PATH = VIDS_DIR['train']

diautt = random.choice([foo for foo, bar in datasets['train'].items()])
annot = datasets['train'][diautt]
print(annot)
vid_path = os.path.join(VIDEOS_PATH, diautt) +  '.mp4'

mp4 = open(vid_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width=1280 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

{'SrNo': '6423', 'Utterance': 'Yes, that’s very good. Now a uh, big part of this job is cultivating personal relationships, especially with designers.', 'Speaker': 'Mr. Posner', 'Emotion': 'neutral', 'Sentiment': 'neutral', 'Dialogue_ID': '640', 'Utterance_ID': '12', 'Season': '4', 'Episode': '9', 'StartTime': '00:07:13,182', 'EndTime': '00:07:22,941'}


In [4]:
from cltl_face_all.face_alignment import FaceDetection
from cltl_face_all.arcface import ArcFace
from cltl_face_all.agegender import AgeGender

fd = FaceDetection(device='cuda', face_detector='sfd')
ag = AgeGender(device='cpu')
af = ArcFace(device='cpu')

[*] load ckpt from /usr/local/lib/python3.6/dist-packages/cltl_face_all/arcface/./pretrained_models/arc_res50/e_8_b_40000.ckpt


In [7]:
import av
import numpy as np
import os

import signal
import time
from tqdm.notebook import tqdm

class Timeout(Exception):
    pass

def handler(sig, frame):
    raise Timeout

signal.signal(signal.SIGALRM, handler) 


for DATATYPE in tqdm(['train', 'dev', 'test']):
    os.makedirs(os.path.join(SAVE_AT, DATATYPE), exist_ok=True)

    for diautt, anno in tqdm(datasets[DATATYPE].items()):
        signal.alarm(20)
        try:
            to_dump = {}
            vidpath = os.path.join(VIDS_DIR[DATATYPE], diautt) + '.mp4'
            container = av.open(vidpath)
            save_full_path = os.path.join(SAVE_AT, DATATYPE, diautt) + '.npy'
        except Exception as e:
            print(e)
            continue

        if os.path.isfile(save_full_path) and os.path.getsize(save_full_path) > 512:
            continue

        for idx, frame in enumerate(container.decode(video=0)):
            signal.alarm(20)
            try:
                to_dump[idx] = []
                numpy_RGB = np.array(frame.to_image())
                batch = numpy_RGB[np.newaxis, ...]
                bboxes = fd.detect_faces(batch)
                # print(f"number of faces detected in the frame {idx} is {len(bboxes[0])}")

                if len(bboxes[0]) == 0:
                    continue

                landmarks = fd.detect_landmarks(batch, bboxes)

                faces = fd.crop_and_align(batch, bboxes, landmarks)
                faces = np.concatenate(faces, axis=0)
                embeddings = af.predict(faces)
                ages, genders = ag.predict(faces)

                # print(len(bboxes[0]), len(landmarks[0]), len(ages), len(genders), len(embeddings), len(faces))
                for bb, lm, a, g, emb in zip(bboxes[0], landmarks[0], ages, genders, embeddings):
                    x1, y1, x2, y2, prob = bb

                    to_append = {'bbox': bb, 
                                'landmark': lm, 
                                'age': a,
                                'gender': g,
                                'embedding': emb}

                    to_dump[idx].append(to_append)
                # print(len(to_dump), idx+1)
            except Exception as e:
                print(e)
                continue

        np.save(save_full_path, to_dump)

/usr/local/lib/python3.6/dist-packages/cltl_face_all/face_alignment/matlab_cp2tform.py:84: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  r, _, _, _ = lstsq(X, U)


KeyboardInterrupt: ignored